In [ ]:
# =============================================
# Generic p-out-of-q experiment (ARM, macro-MLP)
# - per-run: print R2OOS by maturity (+ mean)
# - final : pick top-p by mean R2OOS, print per-maturity average
# =============================================
import os, sys, re, time, warnings, argparse
from typing import Dict, Any, List
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

from rolling_framework import Machine  # framework entrypoint

# ---------------------- CLI ----------------------
TOP_P = 3
TOTAL_Q = 5
if TOP_P > TOTAL_Q:
    sys.exit("[ERROR] p must be <= q")

# ---------------------- Config -------------------
DATA_DIR   = "data/"
Y_FILE     = os.path.join(DATA_DIR, "exrets.csv")
SLOPE_FILE = os.path.join(DATA_DIR, "slope.csv")
YL_FILE    = os.path.join(DATA_DIR, "yl_all.csv")
MACRO_FILE = os.path.join(DATA_DIR, "MacroFactors.csv")

BURN_START, BURN_END     = "197108", "199001"
PERIOD_START, PERIOD_END = "197108", "202312"
HORIZON = 12
MATURITIES = ["xr_2","xr_3","xr_5","xr_7","xr_10"]

# ---------------------- Helpers ------------------
def _load_csv(path, name):
    try:
        return pd.read_csv(path, index_col="Time")
    except FileNotFoundError as e:
        sys.exit(f"[ERROR] missing {name} → {e.filename}")

def _align_time(*dfs):
    idx=None
    for d in dfs:
        idx = d.index if idx is None else idx.intersection(d.index)
    return [d.loc[idx].sort_index() for d in dfs]

def _direct_pairs(slope_cols, y_cols):
    mk = lambda s: re.search(r"(\d+)", s).group(1) if re.search(r"(\d+)", s) else None
    y_map = {mk(c): c for c in y_cols}
    return [(sc, y_map[mk(sc)]) for sc in slope_cols if mk(sc) in y_map]

def _series_mean(x) -> float:
    if isinstance(x, pd.Series):
        return float(np.nanmean(x.to_numpy(dtype=float, copy=False)))
    if isinstance(x, pd.DataFrame):
        return float(np.nanmean(x.to_numpy(dtype=float, copy=False)))
    try:
        return float(x)
    except Exception:
        return float("nan")

# ---------------------- Load ---------------------
y     = _load_csv(Y_FILE,   "exrets")
slope = _load_csv(SLOPE_FILE, "slope")
yl    = _load_csv(YL_FILE,   "yl_all")
macro = _load_csv(MACRO_FILE,"MacroFactors")

# targets
y_cols = [c for c in MATURITIES if c in y.columns]
if not y_cols:
    sys.exit("[ERROR] MATURITIES not in exrets")
y = y[y_cols]

# align
y, slope, yl, macro = _align_time(y, slope, yl, macro)
X_macro = pd.concat([slope, macro], axis=1)

print("✓ Loaded data shapes:",
      {k:v.shape for k,v in [("y",y),("slope",slope),("yl",yl),("macro",macro),("X_macro", X_macro)]})
print("✓ p-out-of-q:", TOP_P, "/", TOTAL_Q)

# ---------------------- Model options/grid --------
BASE_OPT = {
    "base_on": True,
    "base_cols":   list(slope.columns),
    "target_cols": list(y.columns),
    "residual_kind": "mlp",
    "feature_cols": list(macro.columns),
    "standardize_res": True,
    "mlp_hidden": (64, 32),
    "mlp_dropout": 0.1,
    "mlp_lr": 1e-3,
    "mlp_wd": 1e-4,   # L2 (weight decay)
    "mlp_epochs": 200,
    "mlp_patience": 20,
    "seed": 0,
}

BASE_GRID = {
    "arm__residual_model__module__hidden": [(16,), (16, 8)],
    "arm__residual_model__module__dropout": [0.2],
    "arm__residual_model__optimizer__lr": [1e-3],
    "arm__residual_model__optimizer__weight_decay": [1e-3],
}

# ---------------------- Run once ------------------
def run_once(seed: int) -> Dict[str, Any]:
    opt = dict(BASE_OPT)
    opt["seed"] = int(seed)
    try:
        import torch
        torch.manual_seed(seed)
    except Exception:
        pass
    np.random.seed(seed)

    m = Machine(
        X_macro, y, "ARM",
        option=opt, params_grid=BASE_GRID,
        burn_in_start=BURN_START, burn_in_end=BURN_END,
        period=[PERIOD_START, PERIOD_END], forecast_horizon=HORIZON
    )
    print(f"\n▶ Run seed={seed}")
    t0 = time.time()
    m.training()
    elapsed = time.time() - t0

    r2 = m.R2OOS()  # expected Series by maturity
    if not isinstance(r2, pd.Series):
        r2 = pd.Series({"_scalar": _series_mean(r2)})
    mean_r2 = _series_mean(r2)

    print("  R2OOS by maturity:")
    print(r2.round(4))
    print(f"  R2OOS mean: {mean_r2:.6f}   elapsed: {elapsed/60:.2f} min")

    return {"seed": seed, "r2": r2, "mean": mean_r2}

# ---------------------- p-out-of-q loop -----------
results: List[Dict[str, Any]] = []
for seed in range(1, TOTAL_Q + 1):
    results.append(run_once(seed))

# table of means
means_df = pd.DataFrame({
    "seed": [r["seed"] for r in results],
    "R2OOS_mean": [r["mean"] for r in results],
}).sort_values("R2OOS_mean", ascending=False).reset_index(drop=True)

print("\n=== Ranking by mean R2OOS (all maturities) ===")
print(means_df)

# pick top-p
top = means_df.head(TOP_P)
top_seeds = top["seed"].tolist()
print(f"\nTop-{TOP_P} seeds:", top_seeds)

# per-maturity average across top-p runs (score-level averaging)
top_r2_list = []
for r in results:
    if r["seed"] in top_seeds:
        top_r2_list.append(r["r2"].reindex(y.columns))

# align index and average
if top_r2_list:
    top_r2_df = pd.concat(top_r2_list, axis=1)
    top_r2_df.columns = [f"seed_{s}" for s in top_seeds]
    avg_top_r2 = top_r2_df.mean(axis=1)

    print("\n=== Per-maturity average R2OOS over top-p runs ===")
    print(avg_top_r2.round(4))
    print(f"\nAverage of the above (grand mean): {float(np.nanmean(avg_top_r2.to_numpy())):.6f}")
else:
    print("\n[WARN] No top-p results aggregated (empty).")

DNN_DUAL rolling:   4%|▍         | 20/520 [03:47<1:34:56, 11.39s/it]
